In [34]:
import numpy as np
import pandas as pd
import math
import os
import pickle
import importlib as il

funcModule = "data_structure"
ds = il.import_module(funcModule)

playerDB = {}

In [51]:
if os.path.exists("gameLogs.csv"):
    rawData = pd.read_csv("gameLogs.csv")
else:
    print("no such file found")

colNames = list(rawData.columns.values)

i = 0

for i in range(len(colNames)):
    colNames[i] = colNames[i].replace(" ","_")

rawData.columns = colNames

rawData.iloc[:3]

,DATA_SET,DATE,PLAYER_FULL_NAME,POSITION,OWN_TEAM,OPP_TEAM,VENUE_(R/H),MIN,FG,FGA,...,FTA,OR,DR,TOT,A,PF,ST,TO,BL,PTS
0,2016-2017 Regular Season,10/25/2016,Carmelo Anthony,SF,New York,Cleveland,R,30.1,8,18,...,2,1,4,5,3,5,1,4,0,19
1,2016-2017 Regular Season,10/25/2016,Kristaps Porzingis,PF,New York,Cleveland,R,32.8,5,13,...,6,4,3,7,0,5,1,2,2,16
2,2016-2017 Regular Season,10/25/2016,Joakim Noah,C,New York,Cleveland,R,19.5,0,1,...,0,1,5,6,3,1,1,1,0,0


In [53]:
playerCounter = 1

for eachName in rawData['PLAYER_FULL_NAME'].unique():
    playerDB[eachName] = ds.player({'name':eachName,'playerID':playerCounter})
    playerCounter += 1

In [54]:
checkPt = 2
i = 0

for eachEntry in rawData.itertuples():
    print(eachEntry)
    if i == checkPt:
        break
    else:
        i += 1

Pandas(Index=0, DATA_SET='2016-2017 Regular Season', DATE='10/25/2016', PLAYER_FULL_NAME='Carmelo Anthony', POSITION='SF', OWN_TEAM='New York', OPP_TEAM='Cleveland', _7='R', MIN=30.100000000000001, FG=8, FGA=18, _11=1, _12=4, FT=2, FTA=2, OR=1, DR=4, TOT=5, A=3, PF=5, ST=1, TO=4, BL=0, PTS=19)
Pandas(Index=1, DATA_SET='2016-2017 Regular Season', DATE='10/25/2016', PLAYER_FULL_NAME='Kristaps Porzingis', POSITION='PF', OWN_TEAM='New York', OPP_TEAM='Cleveland', _7='R', MIN=32.799999999999997, FG=5, FGA=13, _11=3, _12=5, FT=3, FTA=6, OR=4, DR=3, TOT=7, A=0, PF=5, ST=1, TO=2, BL=2, PTS=16)
Pandas(Index=2, DATA_SET='2016-2017 Regular Season', DATE='10/25/2016', PLAYER_FULL_NAME='Joakim Noah', POSITION='C', OWN_TEAM='New York', OPP_TEAM='Cleveland', _7='R', MIN=19.5, FG=0, FGA=1, _11=0, _12=0, FT=0, FTA=0, OR=1, DR=5, TOT=6, A=3, PF=1, ST=1, TO=1, BL=0, PTS=0)
